In [ ]:
import requests
import pandas as pd
import io
import csv

import geopandas as gpd
import configparser
from pathlib import Path

In [ ]:
# Config file contains file paths and name of township (uesd to match parcel TWP_CITY column)
config = configparser.ConfigParser()

# Change this path to the location of your own config.ini file
config.read('/home/petermitchell/Documents/ContractWork/config.ini')
config.sections()

# Set variables to inputs from config file
verified_wells_path = config['DEFAULT']['verified_wells']
unverified_wells_path = config['DEFAULT']['unverified_wells']
parcels_path = config['DEFAULT']['parcels']
samples_path = config['DEFAULT']['samples']
output_path = config['DEFAULT']['output']

output_dir = Path(output_path)
output_dir.mkdir(parents=True, exist_ok=True)

In [ ]:
url = 'https://geocoding.geo.census.gov/geocoder/geographies/addressbatch'
input_path = ''
files = {'addressFile': (input_path, open(input_path, 'rb'), 'text/csv')}
payload = {'benchmark':'Public_AR_Current', 'vintage':'Current_Current'}
print('Posting requests...')
s = requests.post(url, files=files, data=payload)
print(type(s))

In [ ]:
df = pd.read_csv(io.StringIO(s.text), sep=',', header=None, quoting=csv.QUOTE_ALL, keep_default_na=False, names = ['Lab_SampleID', 'address_in', 'match_indicator', 'match_type', 'address_out', 'long_lat', 'tiger_edge', 'street_side', 'FIPS_STATE', 'FIPS_COUNTY', 'CENSUS_TRACT', 'CENSUS_BLOCK'])

In [ ]:
df[['long', 'lat']] = df['long_lat'].str.split(',', expand=True)
out = df[['Lab_SampleID', 'long', 'lat', 'address_in', 'match_indicator', 'match_type', 'address_out']]
out.to_csv(output_dir / '911_addresses_out.csv')


In [ ]:
parcels = gpd.read_file(parcels_path).to_crs(26915)

verified_wells = gpd.read_file(verified_wells_path)
unverified_wells = gpd.read_file(unverified_wells_path)

verified_wells['Verified'] = True
unverified_wells['Verified'] = False

# Join verified and unverified wells into a single dataframe
wells = gpd.GeoDataFrame(pd.concat([verified_wells, unverified_wells], ignore_index=True))

# Export all wells
#wells.to_file(output_dir / "wells.gpkg")

In [ ]:

# Set samples variable to the output of the geocoding process
samples = out

In [ ]:
# Remove ungeocoded samples, give point geometry
samples = samples.replace('', pd.NA).dropna(subset=['lat', 'long'])
located_samples = gpd.GeoDataFrame(samples, geometry=gpd.points_from_xy(samples["long"], samples["lat"], crs="EPSG:4326")).to_crs(26915)
print(len(located_samples))
print(located_samples.crs)

located_samples.to_file("geocoded_samples.gpkg")
located_samples = gpd.sjoin(parcels, located_samples, how="inner", predicate="intersects")
print(len(located_samples), "samples match at least one parcel")

# Set the right coordinate system (NAD27 UTM 15N, EPSG 26915)
located_samples = located_samples.to_crs(26915)
located_samples.to_file(output_dir / 'located_many.gpkg')

# Remove uncessary columns, drop cases where multiple parcels have the same RR address
multiple_parcels = located_samples[located_samples.duplicated(subset=['Lab_SampleID'], keep=False)]

multiple_parcels.to_file(output_dir / 'multiple_parcels.gpkg')
print(multiple_parcels["Lab_SampleID"])
located_samples = located_samples[['Lab_SampleID', 'address_out', 'geometry']].drop_duplicates(subset='Lab_SampleID', keep=False)

print(len(multiple_parcels))


print(len(located_samples), "samples match exactly one parcel")

#TODO: 134 in confidence code vs 291 here


In [ ]:
# Match wells to parcels with associated samples using an intersect test
sample_well_match = gpd.sjoin(located_samples, wells, how="inner", predicate="intersects")
print(len(sample_well_match))
sample_well_match.to_file(output_dir / 'sample_well_match.gpkg')

# Drop rows where the year sampled is before the year drilled
# unknown drill dates have a value of '0' and therefore survive this check

# Check if the sample ID contains a sample year, otherwise skip this check
date_checkable_samples = sample_well_match[sample_well_match['Lab_SampleID'].str.contains('\d{4}-\d{2,3}-\d{4}', regex=True)]
non_date_checkable_samples = sample_well_match[~sample_well_match['Lab_SampleID'].str.contains('\d{4}-\d{2,3}-\d{4}', regex=True)] # There's probably an easier way of just taking the difference between the last df and the df with everything in it
print("Sample date can be checked: ", len(date_checkable_samples))
print("Sample date can't be checked: ", len(non_date_checkable_samples))

sample_well_date_match = date_checkable_samples[date_checkable_samples['Lab_SampleID'].str.slice(0,4) >= date_checkable_samples['DATE_DRLL'].astype(str).str.slice(0,4)]
invalid_dates = date_checkable_samples[date_checkable_samples['Lab_SampleID'].str.slice(0,4) < date_checkable_samples['DATE_DRLL'].astype(str).str.slice(0,4)]
print("Number of samples where sample year was before drill year: ", len(invalid_dates))


passed_samples = pd.concat([sample_well_date_match, non_date_checkable_samples])
print("Number of samples where sample year was equal to or after drill year: ", len(passed_samples))

sample_well_date_match.to_file(output_dir / 'sample_well_match_datechecked.gpkg')

# Filter to only necessary columns and remove cases where there are multiple wells on a parcel
unique_sample_well_match = passed_samples[['Lab_SampleID', 'UTME', 'UTMN', 'WELLID', 'Verified', 'DATE_DRLL', 'geometry']].drop_duplicates(subset='Lab_SampleID', keep=False)
print("Number of samples that match to a single well: ", len(unique_sample_well_match))

In [ ]:

# Export samples that can be confidently matched to a single well
unique_sample_well_match.to_file(output_dir / 'unique_sample_well_match_911.gpkg')
unique_sample_well_match.to_csv(output_dir / 'unique_sample_well_match_911.csv')

print("Matches with at least one well:", len(sample_well_match))
print("Matches with exactly one well:", len(unique_sample_well_match))